# WAZE INTERMEDIATE DATA CLEANING

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [53]:
waze = pd.read_csv('../../data/01_raw/waze.csv')

## Issues discovered during data exploration: 

1. **Country** - There are 15 different unique values listed in the country variables After closer inspection, the 355 entries without with values other than US have NaN values in every row. These rows should be dropped during cleaning.
1. **nTHumbsup** - The vast majority of the column contains 0 values. This does not look like it will add useful information. This will need to be dropped in data cleaning. 
1. **type** - Weatherhazard needs to be broken up into hazard and weather hazard. 
1. **street** - 2% of street names in the dataset are missing (that's over 200K out of 10 Million). 
1. **pubmills** - This needs to be turned into a timestamp ((Unix time – milliseconds since epoch))
1. **scrapedt** - This is the date that the data wasscraped from the site
1. **subtype** - change all nan values to NO_SUBTYPE

In [54]:
waze.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10055207 entries, 0 to 10055206
Data columns (total 17 columns):
country              object
nTHumbsUp            float64
city                 object
reportRating         float64
confidence           float64
reliability          float64
type                 object
uuid                 object
roadType             float64
magvar               float64
subtype              object
street               object
location_x           float64
location_y           float64
pubMillis            float64
reportDescription    object
scrape_dt            object
dtypes: float64(9), object(8)
memory usage: 1.3+ GB


In [56]:
waze.head()

,country,nTHumbsUp,city,reportRating,confidence,reliability,type,uuid,roadType,magvar,subtype,street,location_x,location_y,pubMillis,reportDescription,scrape_dt
0,US,0.0,"Joliet, IL",0.0,0.0,7.0,ROAD_CLOSED,12d90f41-fd58-3d73-9bac-d2d24a4e1dbb,0.0,0.0,ROAD_CLOSED_EVENT,Briggs St,-88.04366,41.54111,1.510536e+12,Road Closed,2017-11-15T09:21:00Z
1,US,0.0,NaN,0.0,3.0,10.0,ROAD_CLOSED,6bd6a1ff-55b3-3e57-8a57-13ef89e4b391,0.0,0.0,ROAD_CLOSED_EVENT,Smith Rd,-88.02769,41.64151,1.504565e+12,Construction,2017-11-15T09:21:00Z
2,US,0.0,"Lemont, IL",0.0,0.0,7.0,ROAD_CLOSED,0e957bc8-cdcc-395e-913c-facbef9a4494,0.0,0.0,ROAD_CLOSED_EVENT,135th St,-88.00416,41.64081,1.510366e+12,Road Closed,2017-11-15T09:21:00Z
3,US,0.0,"Joliet, IL",0.0,0.0,6.0,ROAD_CLOSED,dfa91aba-dc86-365d-b270-0572fca5ccab,0.0,0.0,ROAD_CLOSED_EVENT,Briggs St,-88.04355,41.53795,1.510536e+12,Road Closed,2017-11-15T09:21:00Z
4,US,0.0,"Lemont, IL",0.0,0.0,7.0,ROAD_CLOSED,4e9af18a-fde4-3f5b-9aeb-1d471988b735,0.0,0.0,ROAD_CLOSED_EVENT,135th St,-88.00494,41.64137,1.510366e+12,Road Closed,2017-11-15T09:21:00Z


In [5]:
waze.isnull().sum()

country                  175
nTHumbsUp                  8
city                 2761258
reportRating             355
confidence               355
reliability              355
type                     355
uuid                     355
roadType                 355
magvar                   355
subtype               403764
street                230610
location_x               355
location_y               355
pubMillis                355
reportDescription    8031443
scrape_dt                488
dtype: int64

There are 355 entries that are completely null in every category. I'll drop these entries. 

In [6]:
waze.dropna(subset=['pubMillis'], inplace=True)

In [7]:
waze.isnull().sum()

country                    0
nTHumbsUp                  0
city                 2760903
reportRating               0
confidence                 0
reliability                0
type                       0
uuid                       0
roadType                   0
magvar                     0
subtype               403409
street                230255
location_x                 0
location_y                 0
pubMillis                  0
reportDescription    8031088
scrape_dt                133
dtype: int64

Let's change the remaining subtype nan values to 'NO_SUBTYPE'

In [8]:
waze.subtype.fillna(value='NO_SUBTYPE', inplace=True)

In [9]:
waze.isnull().sum()

country                    0
nTHumbsUp                  0
city                 2760903
reportRating               0
confidence                 0
reliability                0
type                       0
uuid                       0
roadType                   0
magvar                     0
subtype                    0
street                230255
location_x                 0
location_y                 0
pubMillis                  0
reportDescription    8031088
scrape_dt                133
dtype: int64

**CHANGE PUBMILLS TO DATETIME OBJECT**

In [59]:
waze['pubMillis'] = pd.to_numeric(waze['pubMillis'], downcast='integer')

In [60]:
waze.head()

,country,nTHumbsUp,city,reportRating,confidence,reliability,type,uuid,roadType,magvar,subtype,street,location_x,location_y,pubMillis,reportDescription,scrape_dt
0,US,0.0,"Joliet, IL",0.0,0.0,7.0,ROAD_CLOSED,12d90f41-fd58-3d73-9bac-d2d24a4e1dbb,0.0,0.0,ROAD_CLOSED_EVENT,Briggs St,-88.04366,41.54111,1.510536e+12,Road Closed,2017-11-15T09:21:00Z
1,US,0.0,NaN,0.0,3.0,10.0,ROAD_CLOSED,6bd6a1ff-55b3-3e57-8a57-13ef89e4b391,0.0,0.0,ROAD_CLOSED_EVENT,Smith Rd,-88.02769,41.64151,1.504565e+12,Construction,2017-11-15T09:21:00Z
2,US,0.0,"Lemont, IL",0.0,0.0,7.0,ROAD_CLOSED,0e957bc8-cdcc-395e-913c-facbef9a4494,0.0,0.0,ROAD_CLOSED_EVENT,135th St,-88.00416,41.64081,1.510366e+12,Road Closed,2017-11-15T09:21:00Z
3,US,0.0,"Joliet, IL",0.0,0.0,6.0,ROAD_CLOSED,dfa91aba-dc86-365d-b270-0572fca5ccab,0.0,0.0,ROAD_CLOSED_EVENT,Briggs St,-88.04355,41.53795,1.510536e+12,Road Closed,2017-11-15T09:21:00Z
4,US,0.0,"Lemont, IL",0.0,0.0,7.0,ROAD_CLOSED,4e9af18a-fde4-3f5b-9aeb-1d471988b735,0.0,0.0,ROAD_CLOSED_EVENT,135th St,-88.00494,41.64137,1.510366e+12,Road Closed,2017-11-15T09:21:00Z


In [17]:
new_date = []
for i in waze['pubMillis']:
    s = i / 1000.0
    date = datetime.datetime.fromtimestamp(s).strftime('%Y-%m-%d %H:%M:%S.%f')
    new_date.append(date)
waze['pubmills_datetime'] = new_date

In [21]:
waze['pubmills_datetime'] = pd.to_datetime(waze['pubmills_datetime'])

In [33]:
waze_inter = waze.sort_values('pubmills_datetime').reset_index(drop=True)

In [37]:
waze_inter.drop('scrape_dt', axis=1, inplace=True)

**BREAK UP WEATHER HAZARD**

In [39]:
waze_inter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10054852 entries, 0 to 10054851
Data columns (total 17 columns):
country              object
nTHumbsUp            float64
city                 object
reportRating         float64
confidence           float64
reliability          float64
type                 object
uuid                 object
roadType             float64
magvar               float64
subtype              object
street               object
location_x           float64
location_y           float64
pubMillis            int64
reportDescription    object
pubmills_datetime    datetime64[ns]
dtypes: datetime64[ns](1), float64(8), int64(1), object(7)
memory usage: 1.3+ GB


In [42]:
waze_inter.loc[waze_inter['type']=='WEATHERHAZARD'].subtype.unique()

array(['HAZARD_ON_ROAD_POT_HOLE', 'HAZARD_ON_SHOULDER_CAR_STOPPED',
       'HAZARD_ON_ROAD_OBJECT', 'HAZARD_ON_ROAD_CAR_STOPPED',
       'HAZARD_ON_ROAD_CONSTRUCTION', 'HAZARD_WEATHER_FOG',
       'HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT', 'HAZARD_WEATHER_FLOOD',
       'NO_SUBTYPE', 'HAZARD_ON_ROAD_ROAD_KILL',
       'HAZARD_ON_SHOULDER_MISSING_SIGN', 'HAZARD_ON_ROAD',
       'HAZARD_ON_SHOULDER', 'HAZARD_ON_SHOULDER_ANIMALS',
       'HAZARD_WEATHER', 'HAZARD_ON_ROAD_ICE', 'HAZARD_WEATHER_HAIL',
       'HAZARD_WEATHER_HEAVY_SNOW'], dtype=object)

## WEATHER HAZARD BREAKUP

The original dataset lists both weather hazard and other hazards (roadkill, car stopped, etc.) under the same type category. These need to be seperated in order to fix this. 

**WEATHER HAZARD**
* HAZARD_WEATHER_FOG
* HAZARD_WEATHER_FLOOD
* HAZARD_WEATHER
* HAZARD_ON_ROAD_ICE
* HAZARD_WEATHER_HAIL
* HAZARD_WEATHER_HEAVY_SNOW

**HAZARD OTHER**
* HAZARD_ON_ROAD_POT_HOLE
* HAZARD_ON_SHOULDER_CAR_STOPPED
* HAZARD_ON_ROAD_OBJECT
* HAZARD_ON_ROAD_CAR_STOPPED 
* HAZARD_ON_ROAD_CONSTRUCTION
* HAZARD_ON_ROAD_TRAFFIC_LIGHT_FAULT
* HAZARD_ON_ROAD_ROAD_KILL
* HAZARD_ON_SHOULDER_MISSING_SIGN
* HAZARD_ON_ROAD
* HAZARD_ON_SHOULDER
* HAZARD_ON_SHOULDER_ANIMALS
* NO_SUBTYPE

In [47]:
waze_inter.to_csv('../../data/02_intermediate/waze_intermediate.csv', index=False)